# Text Classification With BERT and KerasNLP

Now since I am done building the sentiment analysis model using different algorithms, I will make use of BERT, a popular Masked Language Model which is bidirectional (it has access to the words left and right) to build a the text classification model and also KerasNLP, which provides a simple Keras API for training and finetuning NLP models to classify the sentiments.

In [83]:
# import the required libraries

import pandas as pd
import numpy as np
import re
import string
import tensorflow as tf
from tensorflow import keras
import keras_nlp
from transformers import BertTokenizer, TFBertForSequenceClassification
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split

In [84]:
# load the exported data
df1 = pd.read_csv('exported_sentiments.csv')

In [85]:
# encode the target labels
df1['Sentiments'] = df1['Sentiments'].replace({
    'negative': 0,
    'positive': 1
})
df1['Sentiments'].value_counts()

0    59
1    41
Name: Sentiments, dtype: int64

In [86]:
X = df1['Feedback']
y = df1['Sentiments']

In [137]:
print(y)
print()
X.to_frame()

0     1
1     1
2     0
3     0
4     0
     ..
95    1
96    0
97    0
98    1
99    1
Name: Sentiments, Length: 100, dtype: int64



,Feedback
0,"The man is too fast in his teaching,he clearly..."
1,The class is dry but he really puts in efforts
2,The course is shit and it's a threat to my bra...
3,"He no try at all, didn’t teach well."
4,Ogbeni you sef know as e dae go
...,...
95,easy and no wahala
96,terrible way of teaching with the I-dont-care ...
97,do not like coding
98,this practical is hard on top 1 unit course haba


In [87]:
# Text Preprocessing of the texts column using NLTK
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"http\S+|www\S+|@\w+|#\w+", "", text)
    text = re.sub(r"[^\w\s]", "", text)
    text = re.sub(r'\b[0-9]+\b\s*', '', text)
    text = ''.join([char for char in text if char not in string.punctuation])
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(tokens)

X_preprocessed = [preprocess_text(text) for text in X]

# Split the preprocessed data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.25)

In [88]:
X_train, X_test, y_train, y_test = train_test_split(pd.Series(X_preprocessed), y, test_size=0.25)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(75,) (75,)
(25,) (25,)


In [89]:
# Convert labels to one-hot encoded format
y_train = tf.keras.utils.to_categorical(y_train, num_classes=2, dtype='float32')
y_test = tf.keras.utils.to_categorical(y_test, num_classes=2, dtype='float32')

In [90]:
# load the pretrained BERT model that has been finetuned for sentiment analysis

model_name = "bert_tiny_en_uncased_sst2"
classifier = keras_nlp.models.BertClassifier.from_preset(
    model_name,
    num_classes=2,
    load_weights = True,
    activation='sigmoid' # for the binary classification task
)

The next step is to compile and train the model. The aim here is to use the pre-trained model and finetune it on the dataset.

In [91]:
classifier.compile(
    loss=keras.losses.BinaryCrossentropy(),
    optimizer=keras.optimizers.Adam(),
    jit_compile=True,
     metrics=["accuracy"],
)
# Access backbone programatically (e.g., to change `trainable`).
classifier.backbone.trainable = False
# Fit again.
classifier.fit(x=X_train, y=y_train, validation_data=(X_test,y_test), batch_size=32)

3/3 [==============================] - 54s 15s/step - loss: 0.5175 - accuracy: 0.7600 - val_loss: 0.2971 - val_accuracy: 0.8400


In [92]:
# evaluate the model on the testing data
classifier.evaluate(X_test, y_test,batch_size=32)

1/1 [==============================] - 2s 2s/step - loss: 0.2971 - accuracy: 0.8400


[0.29710203409194946, 0.8399999737739563]

In [129]:
# checking the model to see performance on new samples
sentiment_categories = ["negative", "positive"]

new_examples = list(df1['Feedback'].sample(10))

scores = classifier.predict([preprocess_text(example) for example in new_examples])

for i, score in enumerate(scores):
    print(f"{new_examples[i]}:➡ {sentiment_categories[np.argmax(score)]} with a { (100 * np.max(score)).round(2) } percent confidence.")
    print()

1/1 [==============================] - 2s 2s/step
Lecturer is good but the class is dry and equations are much:➡ positive with a 74.52 percent confidence.

love to code and course is about coding. A plus for me:➡ positive with a 61.06 percent confidence.

The lecturer is good and his course is also good:➡ positive with a 83.17 percent confidence.

Me wey be senior dev con dey struggle for LA class ha:➡ negative with a 90.4 percent confidence.

We are basically left to go study that course on our own. :➡ negative with a 88.04 percent confidence.

Applied my math's knowledge from 200L for the most part of course:➡ negative with a 67.25 percent confidence.

stupid :➡ negative with a 91.4 percent confidence.

stress no dey the course:➡ negative with a 88.66 percent confidence.

very awful course:➡ negative with a 91.96 percent confidence.

Omo, God will judge AK sha.
You don’t take a class like that and expect the students to understand what you are doing, that’s him and God ahead.
Make I 

### Improving model accuracy

In [159]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Define a learning rate scheduler
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

# During model fitting
classifier.fit(x=X_train, y=y_train, validation_data=(X_test, y_test), batch_size=32, callbacks=[lr_scheduler])

3/3 [==============================] - 18s 7s/step - loss: 0.3030 - accuracy: 0.8933 - val_loss: 0.2270 - val_accuracy: 0.9600 - lr: 0.0010


In [160]:
# evaluate the model on the testing data
classifier.evaluate(X_test, y_test,batch_size=32)

1/1 [==============================] - 3s 3s/step - loss: 0.2270 - accuracy: 0.9600


[0.22701358795166016, 0.9599999785423279]

In [162]:
# checking the model to see performance on new samples
sentiment_categories = ["negative", "positive"]

new_examples = list(df1['Feedback'].sample(10))

scores = classifier.predict([preprocess_text(example) for example in new_examples])

for i, score in enumerate(scores):
    print(f"{new_examples[i]}:➡ {sentiment_categories[np.argmax(score)]} with a { (100 * np.max(score)).round(2) } percent confidence.")
    print()

1/1 [==============================] - 3s 3s/step
Basically from my own experience, CPE 321 isn’t hard but the method employed by the lecturer in teaching the course leaves much to be desired. I would have loved if the lecturer took his time and explained the course very well. :➡ positive with a 78.34 percent confidence.

love to code:➡ positive with a 82.69 percent confidence.

The lecturer is good and his course is also good:➡ positive with a 93.0 percent confidence.

The course is shit and it's a threat to my brain,the teaching mode is so poor :➡ negative with a 92.5 percent confidence.

relatively easy:➡ positive with a 83.17 percent confidence.

terrible way of teaching with the I-dont-care attitude:➡ negative with a 92.58 percent confidence.

very awful course:➡ negative with a 92.63 percent confidence.

course should  not be difficult but lecturer messed it up:➡ negative with a 90.59 percent confidence.

The lecturer is good, I like him.:➡ positive with a 92.95 percent confidenc

In [163]:
# # Set some layers of the BERT backbone to trainable
# classifier.backbone.layers[-3:].trainable = True

# # Compile and fit the model again
# classifier.compile(
#     loss=keras.losses.BinaryCrossentropy(),
#     optimizer=keras.optimizers.Adam(learning_rate=1e-5),  # Adjust the learning rate
#     metrics=["accuracy"]
# )
# classifier.fit(x=X_train, y=y_train, validation_data=(X_test, y_test), batch_size=32, callbacks=[lr_scheduler])

## Finetune BERT With User-controlled Preprocessing

In [130]:
preprocessor = keras_nlp.models.BertPreprocessor.from_preset(
    model_name,
    sequence_length=128,
)

In [131]:
training_data = tf.data.Dataset.from_tensor_slices(([X_train], [y_train]))
validation_data = tf.data.Dataset.from_tensor_slices(([X_test], [y_test]))

train_cached = (
    training_data.map(preprocessor, tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE)
)
test_cached = (
    validation_data.map(preprocessor, tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE)
)

In [132]:
# Pretrained classifier.
classifier2 = keras_nlp.models.BertClassifier.from_preset(
    model_name,
    preprocessor=None,
    num_classes=2,
    load_weights = True,
    activation='sigmoid'
)
classifier2.compile(
    loss=keras.losses.BinaryCrossentropy(),
    optimizer=keras.optimizers.Adam(),
    jit_compile=True,
     metrics=["accuracy"],
)
classifier2.fit(train_cached, validation_data=test_cached,epochs=10)

Epoch 1/10
1/1 [==============================] - 29s 29s/step - loss: 0.5401 - accuracy: 0.7600 - val_loss: 0.3550 - val_accuracy: 0.8000
Epoch 2/10
1/1 [==============================] - 1s 1s/step - loss: 0.4694 - accuracy: 0.8000 - val_loss: 0.3077 - val_accuracy: 0.8800
Epoch 3/10
1/1 [==============================] - 1s 1s/step - loss: 0.3728 - accuracy: 0.8400 - val_loss: 0.2386 - val_accuracy: 0.9200
Epoch 4/10
1/1 [==============================] - 1s 1s/step - loss: 0.2558 - accuracy: 0.9333 - val_loss: 0.2576 - val_accuracy: 0.9200
Epoch 5/10
1/1 [==============================] - 1s 1s/step - loss: 0.1996 - accuracy: 0.9467 - val_loss: 0.2943 - val_accuracy: 0.8800
Epoch 6/10
1/1 [==============================] - 1s 1s/step - loss: 0.1725 - accuracy: 0.9733 - val_loss: 0.2555 - val_accuracy: 0.9200
Epoch 7/10
1/1 [==============================] - 1s 1s/step - loss: 0.1049 - accuracy: 0.9867 - val_loss: 0.1916 - val_accuracy: 0.9600
Epoch 8/10
1/1 [=======================

In [179]:
# checking the model to see performance on new samples
sentiment_categories = ["negative", "positive"]

new_examples = list(df1['Feedback'].sample(30))

test_data =  preprocessor([preprocess_text(example) for example in new_examples])
scores = classifier2.predict(test_data)

for i, score in enumerate(scores):
    print(f"{new_examples[i]}:➡ {sentiment_categories[np.argmax(score)]} with a { (100 * np.max(score)).round(2) } percent confidence.")
    print()

1/1 [==============================] - 0s 122ms/step
one-unit course wey get problem. nonsense:➡ negative with a 96.11 percent confidence.

The outline of the course is difficult and lecturer is bad:➡ negative with a 96.07 percent confidence.

This lecturer really tried for us to be honest but with the teaching of hundreds of students at a time, the class is very rowdy and students are mostly left to study on their own.:➡ positive with a 95.93 percent confidence.

Nice teaching from scientist:➡ positive with a 95.98 percent confidence.

relatively easy:➡ positive with a 95.98 percent confidence.

nice:➡ positive with a 95.96 percent confidence.

Lecturer is good but the class is dry and equations are much:➡ positive with a 95.99 percent confidence.

SImple course:➡ negative with a 95.99 percent confidence.

This one-unit course has got a hell of wahala:➡ negative with a 96.08 percent confidence.

The teaching mode and environment is top tier but the course itself requires deep thinking

In [178]:
# # Define sentiment categories
# sentiment_categories = ["negative", "positive"]

# new_examples = list(df1['Feedback'].sample(30))

# # Initialize a list to store predictions
# predictions = []

# # Preprocess and predict sentiment for each new example
# for example in new_examples:
#     # Preprocess the input data using your custom preprocessing function
#     preprocessed_text = preprocess_text(example)
    
#     # Apply the BERT preprocessor to the preprocessed text
#     test_data = preprocessor([preprocessed_text])
    
#     # Predict sentiment using the classifier
#     scores = classifier2.predict(test_data)
    
#     # Determine sentiment category and confidence
#     prediction = f"{sentiment_categories[np.argmax(scores)]} with a {(100 * np.max(scores)).round(2)} percent confidence."
    
#     # Store the prediction
#     predictions.append((example, prediction))

# # Display examples and predictions
# for example, prediction in predictions:
#     print(f"{example}: ➡ {prediction}")